In [1]:
import spacy
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

### Spacy Word Vector Preprocessing

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(doc).vector for doc in tqdm(data.text)])

100%|█████████████████████████████████████████████████████████████████████████████| 7613/7613 [01:00<00:00, 126.54it/s]


In [3]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import TensorBoard
#try 50% dropout rate for hidden layers

In [4]:
doc_vectors[0].shape

NameError: name 'doc_vectors' is not defined

In [5]:
def ffnn(layers, nodes=128):
    model = Sequential()
    
    #Input Layer
    model.add(Dense(nodes, activation="relu", input_dim=300))
    
    for i in range(layers):
        model.add(Dense(nodes, activation="relu", dropout=0.5))
        
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])
    return model

In [61]:
# models = {}
# nodes = [32, 64, 128]
# for i in range(4):
#     for node in nodes:
#         models[f"{i}_{node}"] = ffnn(i, nodes=node)

In [63]:
# for key, model in models.items(): 
#     tensorboard = TensorBoard(log_dir=f"logs2\{key}")
#     model.fit(doc_vectors, y, batch_size=8, epochs=10, validation_split=0.2, callbacks=[tensorboard], verbose=0)

In [62]:
y = np.array(data.target)

In [6]:
final_model = ffnn(0, nodes=64)
final_model.fit(doc_vectors, y, batch_size=8, epochs=10)

NameError: name 'doc_vectors' is not defined

In [84]:
with nlp.disable_pipes():
    target_vectors = np.array([nlp(doc).vector for doc in tqdm(test.text)])

100%|█████████████████████████████████████████████████████████████████████████████| 3263/3263 [00:26<00:00, 125.32it/s]


In [99]:
pred = final_model.predict(target_vectors)

In [100]:
pred = np.where(pred > 0.5, 1, 0)

In [101]:
pred = pred.ravel()

In [103]:
submission = pd.DataFrame({'id': test.id, 'target': pred})
submission.to_csv('disaster_tweet_ffnn.csv', index=False)

In [ ]:
#Leaderboard Score: 0.80061